# Reconstructing genome-scale metabolic models from genomic data

We have previously seen that genome-scale metabolic models (GEMs) contain the set of all known biochemical reactions of a given organism. Further, these reactions are catalyzed by enzymes, which are encoded by genes. Therefore, we can reconstruct a GEM from genomic data, i.e. the set of genes that are present in the genome of an organism. To this end, we need a biochemical database containing reactions and associated gene protein reaction rules, and a reference sequence database to map our query sequences in the genome. This approach is implemented by the Python tool [CarveME](https://github.com/cdanielmachado/carveme), which we will use in this tutorial to reconstruct several GEMs from a set of metagenome assembled genomes (MAGs) obtained from the [Tara Oceans](http://ocean-microbiome.embl.de/companion.html) database. In particular:

* TARA_ARC_108_MAG_00080, _Alteromonas sp._
* TARA_ARC_108_MAG_00174, _Marinobacter sp._
* TARA_ARC_108_MAG_00201, _Polaribacter sp._
* TARA_ARC_108_MAG_00083, _Sulfitobacter sp._


## CarveMe inputs: peptide sequences and universal biochemical model

CarveMe requires two inputs: a set of peptide sequences and a universal biochemical model. The peptide sequences are aligned against a reference database (with Diamond's [blastx](https://github.com/bbuchfink/diamond)) to match peptide sequences with biiochemical reaction IDs in the universal biochemical database, while the latter is used as a scaffold to generate the GEM of the organism of interest. Briefly, CarveME firstly extracts the subset of reactions in the universal model that were mapped to peptide sequences in the input genome, and then performs a guided [gap-filling](https://academic.oup.com/nar/article/46/15/7542/5042022?login=false) procedure to complete the GEM with missing reactions that are required to sustain growth under a given growth medium.

As an example, here are the first 10 lines of one of the input files we will use in this tutorial. It corresponds to the predicted gene sequences (translated) obtained from one of the MAGs (TARA_ARC_108_MAG_00080, corresponding to genus _Alteromonas_) using [Eggnog-mapper](http://eggnog-mapper.embl.de/):

In [15]:
%%bash

head -n 10 ../data/genomes/pseudo-nitzschia/TARA_ARC_108_MAG_00080.genepred.fasta

>TARA_ARC_108_MAG_00080_000000000001_1 # 175 # 588 # -1 # ID=1_1;partial=00;start_type=ATG;rbs_motif=GGA/GAG/AGG;rbs_spacer=5-10bp;gc_cont=0.420
MDSKAPKALLKAQKLANLLDTAVKLPIIPIRIGLDSIVGLIPGAGDALMLLVSLRIVWLG
KSLGMPSALVAQMVKNSAIDFGLGFVPFIGDIVDVFYKANQKNVRLMEKWWISENKADVD
AQTQKKLTEWEKKLDQQ*
>TARA_ARC_108_MAG_00080_000000000001_2 # 625 # 1704 # -1 # ID=1_2;partial=00;start_type=ATG;rbs_motif=GGA/GAG/AGG;rbs_spacer=5-10bp;gc_cont=0.444
MKFKSNQGFTHSQSDKIGVLVTNLGTPESPTAAALRPYLKEFLSDPRVVEIPRALWWFIL
NLIILNTRPKRSAEAYKTVWTEEGSPLLTITKSQAKAIEARCKAEYGDDVVVDFAMRYGN
PAISDTIERMLSQGVRKLVVLPLYPQYSASTTASTFDAIAKDFTKRRWLPELRFVNHYND
RPDYIKALANKVRAYWEEHGKADKLILSYHGIPKRYLLNGDPYHCECHKTSRLLAEELGL
THEQYMTTFQSRFGKAEWLKPYTDETMKSLPGNGVKSIQVMCPGFSADCLETIEEIGEEN


We will employ a generic metabolic model representativve of a prokaryotic organism as the input universal metabolic model, since all MAGs in this example correspond to prokaryotic organisms. The universal model has been derived from the universal model of the [BIGG](http://bigg.ucsd.edu/) database, and aggregates biochemical reactions from multiple prokaryotic species. Let's load it in `cobrapy` and print its summary:

In [16]:
import cobra
from phycogem.reconstruction_helpers import get_medium_dict_from_media_db, get_dict_of_metabolite_ids


universe = cobra.io.read_sbml_model("../data/carveme_universes/prokaryote_carveme_curated.xml")
universe

Name,bigg_universal
Memory address,7f401d48c610
Number of metabolites,6861
Number of reactions,17851
Number of genes,0
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytoplasm, periplasm, extracellular"


We see that the universal model contains a total of 17851 reactions and 6861 metabolites, three compartments: cytosol, periplasm and extracellular, and a _Growth_ pseudo-reactions, which is tailored to prokaryotic organisms.

## Carving models

We are now ready to start reconstructing GEMs for each of the input MAGs with CarveME. Note that we need to define a medium for the reconstruction, as mentioned above, this is needed for the gap-filling step, as the procedure adds missing reactions to guarantee that the reconstructed model can sustain growth in the defined medium. In this example, we will use a simple marine medium containing inorganic components, some vitamins and cofactors and a carbon source:

In [17]:
medium_id = "MARINE"
met_names = get_dict_of_metabolite_ids("../data/compounds/BIGG_metabolites.json")

medium = get_medium_dict_from_media_db(
    "../data/marine_media/media_db.tsv",
    medium_id
)

for rxn_ex in medium:
    met_id = "_".join("_".join(rxn_ex.split("_")[1:]).split("_")[:-1])
    print(f"{met_id}: {met_names[met_id] if met_id in met_names else met_id}")

glyb: Glycine betaine
btn: Biotin
ca2: Calcium
co: Carbon monoxide
co2: CO2 CO2
cobalt2: Co2+
cu2: Copper
fe2: Fe2+ mitochondria
fe3: Iron (Fe3+)
h2: Hydrogen
h2o: H2O H2O
hco3: Bicarbonate
k: Potassium
mg2: Magnesium
mn2: Manganese
n2: Nitrogen
na1: Sodium
nh4: Ammonium
no2: Nitrite
no3: Nitrate
o2: O2 O2
photon: Light
pi: Phosphate
h: H+
so4: Sulfate
urea: Urea CH4N2O
zn2: Zinc
cl: Chloride
thm: Thiamin
fol: Folate
adodbl: adodbl
cbl1: Cob(I)alamin
glc__D: D-Glucose


In [18]:
%%bash

GENOME_DIR="../data/genomes/pseudo-nitzschia/"

for genome_file in "${GENOME_DIR}"*.fasta; do
    base_name=$(basename "$genome_file" .fasta)
    echo "Running $base_name"
    carve \
        --universe-file "../data/carveme_universes/prokaryote_carveme_curated.xml" \
        --solver gurobi \
        -o "../results/models/${base_name}.xml" \
        --init MARINE \
        --gapfill MARINE \
        --mediadb "../data/marine_media/media_db.tsv" \
        --fbc2 \
        "$genome_file" >/dev/null 2>&1
done

Running TARA_ARC_108_MAG_00080.genepred
Running TARA_ARC_108_MAG_00083.genepred
Running TARA_ARC_108_MAG_00117.genepred
Running TARA_ARC_108_MAG_00139.genepred
Running TARA_ARC_108_MAG_00174.genepred
Running TARA_ARC_108_MAG_00179.genepred
Running TARA_ARC_108_MAG_00201.genepred


## Loading the reconstructed models in cobrapy

Let's take a look at the reconstructed models in cobrapy. We will load them and optimize growth using the default flux bounds that are built into the model.

In [19]:
models = {}
models["Sulfitobacter"] = cobra.io.read_sbml_model(
    "../results/models/TARA_ARC_108_MAG_00083.genepred.xml"
    )
models["Polaribacter"] = cobra.io.read_sbml_model(
    "../results/models/TARA_ARC_108_MAG_00201.genepred.xml"
    )
models["Alteromonas"] = cobra.io.read_sbml_model(
    "../results/models/TARA_ARC_108_MAG_00080.genepred.xml"
    )
models["Marinobacter"] = cobra.io.read_sbml_model(
    "../results/models/TARA_ARC_108_MAG_00174.genepred.xml"
    )

In [20]:
for genus, model in models.items():
    max_bio = model.slim_optimize()
    print(f"{genus}")
    print(f"Reactions: {len(model.reactions)} | Metabolites: {len(model.metabolites)} | Max Growth: {max_bio:.2f} 1/h")

Sulfitobacter
Reactions: 2752 | Metabolites: 1825 | Max Growth: 0.34 1/h
Polaribacter
Reactions: 1909 | Metabolites: 1300 | Max Growth: 0.58 1/h
Alteromonas
Reactions: 2990 | Metabolites: 2013 | Max Growth: 1.76 1/h
Marinobacter
Reactions: 3231 | Metabolites: 2085 | Max Growth: 1.44 1/h


## Merging the reconstructed models into a community (meta)model

CarveME provides the utility function `merge_community`, which allows to merge multiple GEMs into a single community model. A community model is formed by a collection of individual genome-scale models, each one with its own set of reactions and metabolites, but with shared extracellular compartment and extracellular metabolites. Thus, individual models can interact through extracellular metabolites, which can be exchanged between models. To illustrate this concept, let's consider the following toy example:

![toy community model](images/toy_community_model.png)

The community model above is composed of three individual models, $A - C$, which share extracellular compartment and extracellular metabolites $e_1 - e_3$. Individual models can interact through the exchange of extracellular metabolites, e.g. model $A$ can export metabolite $e_1$ through reaction $v^A_e1$, which, in turn, is imported by model B through reaction $v^B_{e1}$. Note that exchange reactions, such as $EX_{e1}$, represent production or consumption of extracellular metabolites by processes that are independent of the models contained in the community.

A community model is useful to study the metabolic interactions between the different organisms in the community, as well as to investigate how these interactions affect growth. Let's merge the four reconstructed models into a single community model, and then import it into cobrapy:

In [3]:
%%bash

merge_community \
    ../results/models/TARA_ARC_108*.xml \
    -o ../results/merged_community.xml \
    -i M9[marine] \
    --mediadb "../data/marine_media/media_db.tsv" \
    --fbc2

/home/robaina/.local/lib/python3.8/site-packages/reframed/core/environment.py:75: UserWarning: Exchange reaction not in model: R_EX_photon_e
  warn(f'Exchange reaction not in model: {r_id}')
/home/robaina/.local/lib/python3.8/site-packages/reframed/core/environment.py:75: UserWarning: Exchange reaction not in model: R_EX_n2_e
  warn(f'Exchange reaction not in model: {r_id}')


In [2]:
models["community"] = cobra.io.read_sbml_model("../results/merged_community.xml")
models["community"]

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-05


Name,merged_community
Memory address,7f1c394456d0
Number of metabolites,11377
Number of reactions,17668
Number of genes,5363
Number of groups,0
Objective expression,1.0*community_growth - 1.0*community_growth_reverse_7473b
Compartments,"extracellular environment, cytoplasm, periplasm, cytoplasm, periplasm, cytoplasm, periplasm, cytoplasm, periplasm, cytoplasm, periplasm, cytoplasm, periplasm, cytoplasm, periplasm"


We can see that the community model contains a single, shared extracellular compartment and individual cytosolic and periplasmic compartments for each species in the community. Morever, CarveME's `merge_community` has created a _community_ biomass reaction, which is an additive reaction based on the individual biomass reactions of each species in the community. This reaction could be used to simulate the growth of the community as a whole. However, it is advisable to handle with care, since predicting the growth of a community depends on multiple factors, such as the relative abundance of each species in the community, the metabolic interactions between them, environmental conditions, etc. For our purposes, we can ignore the community biomass reaction, and instead, use the individual biomass reaction of each species to ensure positive growth of each species in the community.

As we saw before, the community model contains a shared extracellular space, thus a single exchange reaction for each extracellular metabolite. We can think of each of these exchanges as reactions that represent production and consumption processes of each extracellular metabolite that do not involve any of the species in the community model. Each model, however, contains its own transport reactions from the extracellular space to the cytosol, which represent the uptake of extracellular metabolites by each species. Let's use CO<sub>2</sub> as an example to illustrate this:

Here is the single exchange reaction for CO<sub>2</sub> in the community model:

In [6]:
EX_co2 = [rxn for rxn in models["community"].exchanges if "co2" in rxn.id.lower()][0]
EX_co2

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


Reaction identifier,EX_co2_e
Name,R_EX_co2_e
Memory address,0x7f1c2ba87f50
Stoichiometry,co2_e <=> CO2 CO2 <=>
GPR,
Lower bound,-10.0
Upper bound,1000.0


And here all the individual reactions importing/exporting CO<sub>2</sub> from the extracellular space to the periplasm for each species in the community:

In [10]:
co2_transports = [rxn for rxn in models["community"].reactions if "CO2tex" in rxn.id]
for rxn in co2_transports:
    print(rxn.id, rxn.reaction)

CO2tex_TARA_ARC_108_MAG_00080_genepred co2_e <=> co2_p_TARA_ARC_108_MAG_00080_genepred
CO2tex_TARA_ARC_108_MAG_00083_genepred co2_e <=> co2_p_TARA_ARC_108_MAG_00083_genepred
CO2tex_TARA_ARC_108_MAG_00117_genepred co2_e <=> co2_p_TARA_ARC_108_MAG_00117_genepred
CO2tex_TARA_ARC_108_MAG_00139_genepred co2_e <=> co2_p_TARA_ARC_108_MAG_00139_genepred
CO2tex_TARA_ARC_108_MAG_00179_genepred co2_e <=> co2_p_TARA_ARC_108_MAG_00179_genepred


In [11]:
co2_transports[0]

Reaction identifier,CO2tex_TARA_ARC_108_MAG_00080_genepred
Name,CO2 transport via diffusion (extracellular to periplasm)
Memory address,0x7f1c3638b810
Stoichiometry,co2_e <=> co2_p_TARA_ARC_108_MAG_00080_genepred CO2 CO2 <=> CO2 CO2
GPR,spontaneous_TARA_ARC_108_MAG_00080_genepred
Lower bound,-1000.0
Upper bound,1000.0
